# Whole pipeline sequencially

In [12]:
%load_ext autoreload
%autoreload 2

1. Load the data


In [2]:
from DocumentLoader import arxiv_loader
docs=arxiv_loader(query="1706.03762", only_summary=False)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
docs[0].model_dump()

{'id': None,
 'metadata': {'Published': '2023-08-02',
  'Title': 'Attention Is All You Need',
  'Authors': 'Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin',
  'Summary': 'The dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks in an encoder-decoder configuration. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer, based\nsolely on attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation tasks show these models to be\nsuperior in quality while being more parallelizable and requiring significantly\nless time to train. Our model achieves 28.4 BLEU on the WMT 2014\nEnglish-to-German translation task, improving over the existing best results,\nincluding ensembles by over 2 BLEU. On the WMT 2014 English-

In [4]:
docs[0].page_content

'Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗†\nUniversity of Toronto\naidan@cs.toronto.edu\nŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrence

2. Split the data


In [5]:
from DocumentChunker import recursive_character_splitter
docs=recursive_character_splitter(document=docs,
                                   chunk_size=500, 
                                   chunk_overlap=100)
docs[:3]

[Document(metadata={'Published': '2023-08-02', 'Title': 'Attention Is All You Need', 'Authors': 'Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin', 'Summary': 'The dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks in an encoder-decoder configuration. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer, based\nsolely on attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation tasks show these models to be\nsuperior in quality while being more parallelizable and requiring significantly\nless time to train. Our model achieves 28.4 BLEU on the WMT 2014\nEnglish-to-German translation task, improving over the existing best results,\nincluding ensembles by over 2 BLEU. On the WMT 2014 English-to-French\ntr

In [6]:
len(docs)

102

3. Embed the data


In [7]:
from DocumentEmbedder import embedder_by_ollama
embedder_ollama=embedder_by_ollama()
embedder_ollama

OllamaEmbeddings(model='llama3.2:1b', base_url=None, client_kwargs={})

In [8]:
# Gettng embeddings
from DocumentEmbedder import embedding_maker

embeddings_ollama=embedding_maker(embedder=embedder_ollama,
                                  documents=docs)

In [13]:
from DocumentEmbedder import embedder_by_openai
embedder_openai=embedder_by_openai(model="text-embedding-3-large")
embedder_openai

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x000001F52DB0AC10>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x000001F52EFE1850>, model='text-embedding-3-large', dimensions=1536, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [14]:
# Gettng embeddings
from DocumentEmbedder import embedding_maker

embeddings_openai=embedding_maker(embedder=embedder_openai,
                                  documents=docs)

In [11]:
len(embeddings_ollama), len(embeddings_ollama[0])

(102, 2048)

In [15]:
len(embeddings_openai), len(embeddings_openai[0])

(102, 1536)

4. Create vectore stores

In [16]:
# creating vector store db using already created embeddings.

from VectorStorage import create_chroma_db, create_faiss_db, generic_similerty_search
vectorstoredb_faiss=create_faiss_db(documents=docs,
                               embedder=embedder_openai,
                               embeddings=embeddings_openai,
                               from_embeddings=True)
vectorstoredb_chroma=create_chroma_db(documents=docs,
                               embedder=embedder_openai,
                               embeddings=embeddings_openai,
                               from_embeddings=True)

In [10]:
# vectorstoredb_chroma._embedding_function, vectorstoredb_faiss.embedding_function

In [17]:
query="what is this paper about?"

result=vectorstoredb_faiss.similarity_search(query=query,
                                             k=3)
result

[Document(metadata={'Published': '2023-08-02', 'Title': 'Attention Is All You Need', 'Authors': 'Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin', 'Summary': 'The dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks in an encoder-decoder configuration. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer, based\nsolely on attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation tasks show these models to be\nsuperior in quality while being more parallelizable and requiring significantly\nless time to train. Our model achieves 28.4 BLEU on the WMT 2014\nEnglish-to-German translation task, improving over the existing best results,\nincluding ensembles by over 2 BLEU. On the WMT 2014 English-to-French\ntr

In [18]:
result=vectorstoredb_chroma.similarity_search(query=query,
                                              k=3)
result

[Document(metadata={'Authors': 'Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin', 'Published': '2023-08-02', 'Summary': 'The dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks in an encoder-decoder configuration. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer, based\nsolely on attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation tasks show these models to be\nsuperior in quality while being more parallelizable and requiring significantly\nless time to train. Our model achieves 28.4 BLEU on the WMT 2014\nEnglish-to-German translation task, improving over the existing best results,\nincluding ensembles by over 2 BLEU. On the WMT 2014 English-to-French\ntranslation task, our model establishes 

In [19]:
# Saving vector stores
from VectorStorage import save_vectorstorage
save_vectorstorage(vectorstoredb_chroma, "chroma_db_")
save_vectorstorage(vectorstoredb_faiss, "faiss_db_")

c:\Users\HomePC\DataScience\github-repos\full-fledge-rag\app\VectorStorage.py:57: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstorage.persist()


In [20]:
# Loading vector stores
from VectorStorage import load_vectorstorage
vectorstoredb_faiss=load_vectorstorage(filename="faiss_db_",
                   embedder=embedder_openai,
                   vectordbtype="FAISS")
vectorstoredb_chroma=load_vectorstorage(filename="chroma_db_",
                   embedder=embedder_openai,
                   vectordbtype="Chroma")

c:\Users\HomePC\DataScience\github-repos\full-fledge-rag\app\VectorStorage.py:68: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstorage=Chroma(persist_directory=filename,


5. Creating custom prompts

In [21]:
from CustomPrompts import custom_prompt

prompt = "You are a helpful assistant. You are given the following pieces of retrieved context: {context}. \
The user asked the following question: {question}, only answer from retrieved context. if there is nothing in there, just say IDK."

prompt_template=custom_prompt(template=prompt)

6. LLM selection

In [22]:
from LlmSelection import llm_by_ollama

llm_ollama = llm_by_ollama()

In [23]:
from LlmSelection import llm_by_openai

llm_openai = llm_by_openai()

In [147]:
from LlmSelection import llms_by_groq
llm_groq = llms_by_groq(model="mixtral-8x7b-32768")

7. Inferencing chain

In [25]:
from InferenceChain import inference_chain_with_prompt_template_rag

chain=inference_chain_with_prompt_template_rag(llm=llm_ollama,
                            vectorstorage=vectorstoredb_faiss,
                            prompt_template=prompt_template)

In [26]:
result = chain.invoke({"query": "Who is the author of this paper?"})
result

{'query': 'Where is the author of this paper?',
 'result': 'The author of this paper is Aidan N. Gomez.',
 'source_documents': [Document(metadata={'Published': '2023-08-02', 'Title': 'Attention Is All You Need', 'Authors': 'Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin', 'Summary': 'The dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks in an encoder-decoder configuration. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer, based\nsolely on attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation tasks show these models to be\nsuperior in quality while being more parallelizable and requiring significantly\nless time to train. Our model achieves 28.4 BLEU on the WMT 2014\nEnglish-to-German transla

In [ ]:
# other ways to to create chains

In [48]:
#  Use one of 'human', 'user', 'ai', 'assistant', or 'system'.
from langchain_core.prompts import ChatMessagePromptTemplate, PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

strparser = StrOutputParser()
pt=PromptTemplate.from_template(prompt)
cpt=ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. You are given the following pieces of retrieved context: {context}. Answer concisely based on these if you do not know just say IDK"),
        ("user", "{user_input}")
    ]
    )

In [64]:
chain = cpt | llm_ollama | strparser

In [65]:
result = chain.invoke({"user_input": "Who is the author of this paper?",
                       "context": "be specific to attention is all you need paper"})
result

"I don't have any information about a specific paper. Can you please provide more context or details about the paper, such as its title or a brief summary? I'll do my best to help you find the answer."

In [51]:
# different prompts templates

In [50]:
from langchain_core.prompts import PromptTemplate

# Simple text-based prompt template for generating recipes
recipe_template = PromptTemplate(
    input_variables=["cuisine", "main_ingredient"],
    template="Create a delicious {cuisine} recipe using {main_ingredient} as the primary ingredient. Include preparation steps and cooking time."
)

# Format the prompt
recipe_prompt = recipe_template.format(
    cuisine="Italian", 
    main_ingredient="eggplant"
)
print(recipe_prompt)
# Output: Create a delicious Italian recipe using eggplant as the primary ingredient. Include preparation steps and cooking time.

Create a delicious Italian recipe using eggplant as the primary ingredient. Include preparation steps and cooking time.


In [51]:
from langchain_core.prompts import ChatMessagePromptTemplate

# Create a system message template
system_template = ChatMessagePromptTemplate.from_template(
    role="system", 
    template="You are an expert {profession} with {years_experience} years of experience."
)

# Create a human message template
human_template = ChatMessagePromptTemplate.from_template(
    role="human", 
    template="What advice can you give about {topic}?"
)

# Format the messages
system_prompt = system_template.format(
    profession="data scientist", 
    years_experience=10
)
human_prompt = human_template.format(topic="machine learning best practices")

print("System Message:", system_prompt.content)
print("Human Message:", human_prompt.content)

System Message: You are an expert data scientist with 10 years of experience.
Human Message: What advice can you give about machine learning best practices?


In [133]:
from CustomPrompts import custom_chat_prompt

# Create a complete chat prompt template
tutor_prompt = custom_chat_prompt(template=[
    ("system", "You are a patient {subject} tutor helping a student understand complex concepts."),
    ("human", "I'm struggling to understand {specific_topic}."),
    ("ai", "Let me explain {specific_topic} step by step. First, {explanation_start}"),
    ("human", "Can you provide an example?"),
    ("ai", "Here's a practical example to help you understand {specific_topic}: {detailed_example}")
])
# Format the entire conversation template
formatted_prompt = tutor_prompt.format_messages(
    subject="physics",
    specific_topic="quantum entanglement",
    explanation_start="quantum entanglement is a phenomenon where two particles become interconnected...",
    detailed_example="Imagine two electrons that always spin in opposite directions, no matter how far apart they are..."
)

In [148]:
from InferenceChain import inference_chain_with_prompt_template

chain = inference_chain_with_prompt_template(llm=llm_groq,
                                     prompt_template=recipe_template,
                                     output_parser=strparser)# | strparser
result=chain.invoke({"cuisine":"Italian",
               "main_ingredient":"eggplant"})
print(result)

Melanzane Parmigiana (Eggplant Parmesan)

Ingredients:
- 2 large eggplants (about 2 pounds)
- 3 cups marinara sauce
- 3 cups shredded mozzarella cheese
- 1 cup grated Parmesan cheese
- 1 cup all-purpose flour
- 4 large eggs
- 3 cups breadcrumbs
- 1 bunch fresh basil leaves
- Salt and pepper to taste
- Olive oil for frying

Preparation:

1. Preheat the oven to 375°F (190°C).

2. Wash the eggplants and cut them into 1/4-inch thick slices. Lay the slices on a paper towel-lined baking sheet and sprinkle both sides with salt. Let them sit for 30 minutes to draw out any bitterness. After 30 minutes, pat the eggplant slices dry with paper towels.

3. In a large skillet, heat enough olive oil over medium heat for shallow frying. While the oil is heating, set up your breading station. In one bowl, place the flour. In another bowl, beat the eggs with a pinch of salt and pepper. In a third bowl, combine the breadcrumbs and grated Parmesan cheese.

4. Coat each eggplant slice first in the flour, s

In [134]:
from InferenceChain import inference_chain_with_prompt_template

chain = inference_chain_with_prompt_template(llm=llm_ollama,
                                     prompt_template=tutor_prompt,
                                     output_parser=strparser)# | strparser
result=chain.invoke(input={"subject":"physics",
    "specific_topic":"quantum entanglement",
    "explanation_start":"quantum entanglement is a phenomenon where two particles become interconnected...",
    "detailed_example":"Imagine two electrons that always spin in opposite directions, no matter how far apart they are..."},
    query="what is quantum entanglement?")

print(result)

 

Let's say we have two electrons, A and B. They're entangled in such a way that if electron A spins clockwise, electron B will instantly spin counterclockwise, regardless of the distance between them.

Now, let's consider another pair of electrons, C and D. These electrons are not entangled with each other; they just happen to be close enough for their spins to be correlated in some way.

If we measure the spin of electron A, it will instantly affect the spin of electron C, even if they're on opposite sides of the universe! This is because the two electrons are connected by a shared quantum state.

This phenomenon might seem weird or even spooky at first, but it's actually a fundamental aspect of how the tiny building blocks of matter interact with each other at the quantum level.


In [40]:
import requests

In [41]:
# # This is a large nested json object and will be loaded as a python dict
# json_data = requests.get("https://api.smith.langchain.com/openapi.json").json()

In [41]:
#from utils.chat_history import get_chat_history
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from services.langchain_components.llm_selection import llms_by_groq
from services.langchain_components.custom_prompts import custom_chat_prompt
from langchain_core.messages import AIMessage, HumanMessage
from services.langchain_components.message_trimmer import trim_tokens_in_messages
from typing import Any


chat_prompt = custom_chat_prompt(template=[
    ("ai", 'You are a helpful assistant'),
    ("human",'My name is Hasan')
])
llm=llms_by_groq()
store={}

def get_session_history(session_id: str, trim_messages: bool= False, **kwargs: Any) -> BaseChatMessageHistory:
    """
    kwargs are passed to trim_tokens_in_messages
    kwargs can be llm: ((list[BaseMessage]) -> int) | ((BaseMessage) -> int) | BaseLanguageModel,
    max_tokens: int = 1000,
    strategy: str = "last",
    include_system: bool = True,
    allow_partial: bool = False,
    start_on: str = "human"
    """
    if session_id not in store:
        store[session_id] = ChatMessageHistory()

    # Trim messages in place
    if trim_messages:
        trimmed = trim_tokens_in_messages(messages=store[session_id].messages,
                        **kwargs
                        )
        store[session_id].messages = trimmed  # Update the messages directly

    return store[session_id]

In [65]:
from functools import partial

message_history = RunnableWithMessageHistory(runnable=llm,
                                             get_session_history=partial(get_session_history,session_id='chat_4',trim_messages=True,max_tokens=250,llm=llm))

In [45]:
# session_id=get_session_history('chat_3')
# session_id.messages

[]

In [67]:
store['chat_4']

InMemoryChatMessageHistory(messages=[HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}), AIMessage(content="I don't have the ability to know your name unless you tell me. I can only provide information and answer questions to the best of my knowledge based on the data I have been trained on.", additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run-88336b1e-c99a-4a52-b725-92467b93ee14-0', usage_metadata={'input_tokens': 13, 'output_tokens': 41, 'total_tokens': 54})])

In [54]:
config1 = {"session_id": 'chat_1'}
config2 = {"session_id": 'chat_2'}
config3 = {"session_id": 'chat_3'}
config4 = {"session_id": 'chat_4'}

In [66]:
message_history.invoke(
    [
    ("ai", 'You are a helpful assistant'),
    ("human",'My name is Hasan')
],
    config=config4
)

TypeError: get_session_history() got multiple values for argument 'session_id'

In [56]:
message_history.invoke(
    [
    ("human", 'What is my name?')
],
    config=config4
)

AIMessage(content="You mentioned earlier that your name is Hasan. Is there a reason you are asking me this question again? Is there something you would like to know or discuss related to your name? I'm here to help and have a conversation with you. Just let me know what's on your mind.", additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run-0f8c8756-b9a0-46f3-8eff-633c3f1e93df-0', usage_metadata={'input_tokens': 71, 'output_tokens': 62, 'total_tokens': 133})

In [57]:
message_history.invoke(
    [
    HumanMessage(content='Yes that is correct thank you for remembering my name.')
],
    config=config4
)

AIMessage(content="Of course, I'm here to help and have a conversation with you. It's important to me to remember and use the correct name when addressing someone. I'm glad that I was able to remember your name, Hasan. Is there something specific you would like to talk about or ask me? I'm here to help and have a conversation with you. Just let me know what's on your mind.", additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run-0ed5a63f-81e6-4a84-817a-70e90b12e2e9-0', usage_metadata={'input_tokens': 151, 'output_tokens': 88, 'total_tokens': 239})

In [59]:
message_history.invoke(
    [
    HumanMessage(content="I'm a data scientist.")
],
    config=config4
)

AIMessage(content="That's great! As a data scientist, you have a valuable skill set and are able to use data and statistical analysis to solve problems and gain insights. Data science is a rapidly growing field with many exciting opportunities for professionals with your skills.\n\nAs a data scientist, you may work with large datasets and use a variety of tools and techniques to analyze and interpret the data. This can include using programming languages like Python or R, as well as statistical analysis and machine learning techniques.\n\nData science has many applications in various industries such as healthcare, finance, technology, and government. It can be used to improve decision making, optimize business processes, and gain a better understanding of customers and market trends.\n\nDo you have any specific questions about data science or your work as a data scientist? I'm here to help and have a conversation with you. Just let me know what's on your mind.", additional_kwargs={}, r

In [61]:
message_history.invoke(
    [
    HumanMessage(content="what are new algorithms in genai?")
],
    config=config4
)

AIMessage(content="There are many new and exciting algorithms being developed in the field of natural language processing (NLP) and artificial intelligence (AI) that are specifically geared towards building more advanced and capable language models for use in general AI.\n\nOne area of focus is on unsupervised learning algorithms, which allow models to learn from large amounts of unlabeled data. This is in contrast to supervised learning algorithms, which require labeled data to train a model. Unsupervised learning algorithms can be particularly useful in NLP, where it can be difficult or time-consuming to manually label data.\n\nAnother area of focus is on transfer learning and fine-tuning, which allow models to learn from one task and then apply that knowledge to a new, related task. This is particularly useful in NLP where there is a lot of data available for some tasks (e.g. language translation) but not as much for others (e.g. sentiment analysis).\n\nThere are also new architectu

In [63]:
message_history.invoke(
    [
    ("human", 'What is my name?')
],
    config=config4
)

AIMessage(content="I don't have the ability to know your name unless you tell me. I can only provide information and answer questions to the best of my knowledge based on the data I have been trained on.", additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run-88336b1e-c99a-4a52-b725-92467b93ee14-0', usage_metadata={'input_tokens': 13, 'output_tokens': 41, 'total_tokens': 54})

In [8]:
# chat history sqlite
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

In [6]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

chat_message_history = SQLChatMessageHistory(
    session_id="test_session_id", connection_string="sqlite:///sqlite.db"
)

chat_message_history.add_user_message("Hello")
chat_message_history.add_ai_message("Hi")

C:\Users\HomePC\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3577: LangChainDeprecationWarning: `connection_string` was deprecated in LangChain 0.2.2 and will be removed in 1.0. Use connection instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
chat_message_history.messages

[HumanMessage(content='Hello', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi', additional_kwargs={}, response_metadata={})]

In [53]:
def create_runnable_with_history_input():
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a helpful assistant."),
            MessagesPlaceholder(variable_name="history"),
            ("human", "{question}"),
        ]
    )

    return  prompt | llm

In [54]:
def create_or_get_chain_with_history():
    return RunnableWithMessageHistory(
        runnable=create_runnable_with_history_input(),
        get_session_history=lambda session_id: SQLChatMessageHistory(
            session_id=session_id, connection_string="sqlite:///sqlite.db"
        ),
        input_messages_key="question",
        history_messages_key="history",
    )

In [55]:
# This is where we configure the session id
config = {"configurable": {"session_id": "<SQL_SESSION_ID>"}}

In [56]:
chain_with_history=create_or_get_chain_with_history()
chain_with_history.invoke({"question": "Hi! I'm Hasan"}, config=config)

AIMessage(content='Hello Hasan! It\'s nice to see you again. Is there something specific you would like to ask or discuss today? I\'m here to help. You can ask me about a wide range of topics, such as general knowledge, science, technology, math, and more. Let me know how I can assist you.\n\nJust to give you an idea of the types of questions I can help with, here are a few examples:\n\n* "Can you explain how photosynthesis works?"\n* "What is the square root of 144?"\n* "What are the capitals of the 50 US states?"\n* "Can you tell me about the life cycle of a butterfly?"\n* "What are some famous paintings by Leonardo da Vinci?"\n\nThese are just a few examples, but you can ask me about almost anything you can think of! Go ahead and ask me your question.', additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run-dcfbdf80-7dd9-44bc-a069-b9ff7b9927a7-0', usage_metadata={'input_tokens': 378, 'output_tokens': 197, 'total_tokens': 575})

In [41]:
chain_with_history.invoke({"question": "Whats my name"}, config=config)

AIMessage(content="As you have mentioned a few times now, your name is Hasan. Is there something specific you would like to know or discuss related to your name or anything else? I'm here to help.", additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run-aa8d8a6b-2079-489c-854f-f60fd5b014f9-0', usage_metadata={'input_tokens': 240, 'output_tokens': 42, 'total_tokens': 282})

In [28]:
import sqlite3

def get_existing_tables():
    conn = sqlite3.connect("sqlite.db")
    cursor = conn.cursor()

    # List all tables
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    conn.close()
    return [table[0] for table in tables]

existing_tables = get_existing_tables()
print(existing_tables)

['message_store']


In [70]:
def get_existing_columns_from_tables():
    conn = sqlite3.connect("sqlite.db")
    cursor = conn.cursor()

    # List all tables
    #get columns names
    cursor.execute("SELECT name FROM pragma_table_info('message_store')")
    columns = cursor.fetchall()

    conn.close()
    return [column[0] for column in columns]

existing_columns = get_existing_columns_from_tables()
print(existing_columns)

['id', 'session_id', 'message', 'topic_name']


In [64]:
def list_existing_sessions():
    conn = sqlite3.connect("sqlite.db")
    cursor = conn.cursor()
    
    # This is a simplified example - the actual table name might vary
    cursor.execute("SELECT DISTINCT session_id FROM message_store")
    sessions = cursor.fetchall()
    
    conn.close()
    return [session[0] for session in sessions]

# Get all existing session IDs
existing_sessions = list_existing_sessions()
print(existing_sessions)

[]


In [62]:
def get_session_topic_name_dict():
    conn = sqlite3.connect("sqlite.db")
    cursor = conn.cursor()
    cursor.execute("SELECT session_id, topic_name FROM message_store")
    for i in cursor.fetchall():
        print(i)

In [76]:
get_session_topic_name_dict()

('<SQL_SESSION_ID_1>', 'My introduction Hasan1')
('<SQL_SESSION_ID_1>', 'My introduction Hasan1')
('<SQL_SESSION_ID_1>', 'My introduction Hasan1')
('<SQL_SESSION_ID_1>', 'My introduction Hasan1')
('<SQL_SESSION_ID_1>', 'My introduction Hasan1')
('<SQL_SESSION_ID_1>', 'My introduction Hasan1')
('<SQL_SESSION_ID_1>', 'My introduction Hasan1')
('<SQL_SESSION_ID_1>', 'My introduction Hasan1')
('<SQL_SESSION_ID_1>', 'My introduction Hasan1')
('<SQL_SESSION_ID_1>', 'My introduction Hasan1')
('<SQL_SESSION_ID_1>', 'My introduction Hasan1')
('<SQL_SESSION_ID_1>', 'My introduction Hasan1')
('<SQL_SESSION_ID_1>', 'My introduction Hasan1')
('<SQL_SESSION_ID_1>', 'My introduction Hasan1')


In [79]:
conn = sqlite3.connect("sqlite.db")
cursor = conn.cursor()
cursor.execute("delete from message_store")
cursor.execute("ALTER TABLE message_store DROP COLUMN topic_name")

In [80]:
conn.commit()
conn.close()

In [1]:
from services.langchain_components.inference_chain import inference_chain_chat_history
from services.langchain_components.inference_chain import inference_chain_chat_history_sqlite


In [6]:
chain=inference_chain_chat_history()


In [10]:
session_id="introductory2"

In [8]:
result = chain.invoke({"question": "I'm Hasan and data scientist"}, config={"configurable": {"session_id": session_id}})
print(result)

content="Hello, Hasan! Yes, data science can be a challenging field, but it's also an exciting and rewarding one. It involves working with large and complex data sets, using advanced statistical techniques, and communicating insights to stakeholders. The field is constantly evolving, with new tools and techniques emerging all the time. However, with your skills and expertise, I'm confident you can tackle any challenges that come your way. Is there a specific problem or topic you would like to discuss or learn more about in data science?" additional_kwargs={} response_metadata={'finish_reason': 'stop'} id='run-ad9d4d80-b4ca-4f23-90c4-2ff407d401e1-0' usage_metadata={'input_tokens': 104, 'output_tokens': 110, 'total_tokens': 214}


In [11]:
result = chain.invoke({"question": "say my name to refer me and tell is my field challenging?"}, config={"configurable": {"session_id": session_id}})
print(result)

content='Hello, user! I believe your field can be challenging, as it often requires problem-solving, critical thinking, and staying up-to-date with new developments. Keep up the great work!\n\n(Note: I do not know your specific field, but my response is based on a general understanding that many professions have challenging aspects.)' additional_kwargs={} response_metadata={'finish_reason': 'stop'} id='run-f506eeb3-6e3b-4bb9-b42e-5bd80ad3e21f-0' usage_metadata={'input_tokens': 36, 'output_tokens': 72, 'total_tokens': 108}


In [8]:
session_id="introductorydummy"
chain=inference_chain_chat_history_sqlite(session_id=session_id,
                                          topic_name='Naam wghera 2',
                                          database_name='memory.db')


In [9]:
result = chain.invoke({"question": "I'm Hasan and ai engineer"}, config={"configurable": {"session_id": session_id}})
print(result)

content='Nice to meet you, Hasan! How can I assist you today?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 30, 'total_tokens': 44, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_3de1288069', 'finish_reason': 'stop', 'logprobs': None} id='run-2bec47fe-0764-4769-a1fd-f8a38b8e051f-0' usage_metadata={'input_tokens': 30, 'output_tokens': 14, 'total_tokens': 44, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [10]:
result = chain.invoke({"question": "say my name to refer me and tell is my field challenging?"}, config={"configurable": {"session_id": session_id}})
print(result)

session_id 'introductorydummy' already exists in the database
content='Hasan, as an AI engineer, your field can be quite challenging due to the rapid advancements in technology, the need for continuous learning, and the complexity of developing effective algorithms. However, it can also be very rewarding and impactful.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 65, 'total_tokens': 112, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bba3c8e70b', 'finish_reason': 'stop', 'logprobs': None} id='run-fda2b68c-aa2b-4b3e-8b3b-954b79bf9516-0' usage_metadata={'input_tokens': 65, 'output_tokens': 47, 'total_tokens': 112, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio'

In [1]:
from services.langchain_components.message_trimmer import trim_tokens_in_messages
from services.langchain_components.llm_selection import llms_by_groq
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

In [4]:
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

print(trim_tokens_in_messages(messages, llms_by_groq(), max_tokens=25))

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}), HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}), AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}), HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}), AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]


In [ ]:
# # messages history with sql and memory

# # chat history sqlite
# from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
# from langchain_core.runnables.history import RunnableWithMessageHistory
# from langchain_community.chat_message_histories import SQLChatMessageHistory
# from services.langchain_components.llm_selection import llms_by_groq
# from langchain_core.runnables.base import RunnableConfig
# from langchain_core.runnables.base import Runnable
# from langchain_core.runnables.utils import ConfigurableFieldSpec
# from langchain_core.runnables.utils import Input, Output
# from typing import Optional, Any, Callable, Sequence
# import sqlite3

# def insert_topic_name(session_id: str, topic_name: str):
#     conn = sqlite3.connect("sqlite.db")
#     cursor = conn.cursor()
#     # create new table topics with session_id, topic_name where session_id is primary key
#     cursor.execute("CREATE TABLE IF NOT EXISTS topics (session_id TEXT PRIMARY KEY, topic_name TEXT)")
#     conn.commit()
#     # insert new row into topics table with session_id and topic_name
#     try:
#         cursor.execute("INSERT INTO topics (session_id, topic_name) VALUES (?, ?)", (session_id, topic_name))
#         conn.commit()
#     except sqlite3.IntegrityError:
#         print(f"session_id '{session_id}' already exists in the database")
#     conn.close()


# def create_runnable_with_history_input() -> Runnable:
#     llm = llms_by_groq()
#     prompt = ChatPromptTemplate.from_messages(
#         [
#             ("system", "You are a helpful assistant. Answer to the point only."),
#             MessagesPlaceholder(variable_name="history"),
#             ("human", "{question}"),
#         ]
#     )
#     return  prompt | llm

# def get_chain_with_history(session_id: str, topic_name: str) -> Runnable:

#     history_chain = RunnableWithHistoryCustom(
#     runnable=create_runnable_with_history_input(),
#     session_id=session_id,
#     topic_name=topic_name,
#     get_session_history=lambda session_id: SQLChatMessageHistory(
#         session_id=session_id, connection="sqlite:///sqlite.db"
#     ),
#     input_messages_key="question",
#     history_messages_key="history",
#     )
#     return history_chain

# class RunnableWithHistoryCustom(RunnableWithMessageHistory):
#     """
#     Custom Runnable with message history that includes session_id and topic_name.
#     """
#     session_id: str
#     topic_name: str

#     def invoke(
#         self,
#         input: Input,
#         config: Optional[RunnableConfig] = None,
#         **kwargs: Optional[Any],
#     ) -> Output:
#         # Insert topic name before invoking
#         insert_topic_name(self.session_id, self.topic_name)
        
#         return self.bound.invoke(
#             input,
#             self._merge_configs(config),
#             **{**self.kwargs, **kwargs},
#         )
    
# if __name__ == "__main__":

#     session_id = "<SQL_SESSION_ID_3>"
#     topic_name = "I'm data scientist hasan"
#     chain_with_history=get_chain_with_history(session_id, topic_name)
#     result = chain_with_history.invoke({"question": "who am i?"}, config={"configurable": {"session_id": session_id}})
#     print(result)




#     #session_topic_name_dict = get_session_topic_name_dict()
#     #print(session_topic_name_dict)

#     # #get values only from topic_name column in database
#     # conn = sqlite3.connect("sqlite.db")
#     # cursor = conn.cursor()
#     # cursor.execute("SELECT topic_name FROM message_store")
#     # topic_names = cursor.fetchall()
#     # print(topic_names)


In [1]:
from services.langchain_components.inference_chain import inference_chain_chat_history, _get_session_history, inference_chain_chat_history_sqlite
from services.langchain_components.llm_selection import llms_by_groq

In [2]:
llm=llms_by_groq()

In [3]:
# This is where we configure the session id
session_id="joey2"
config = {"configurable": {"session_id": session_id}}
trim_control = {"should_trim":True}

In [4]:
chain_hist=inference_chain_chat_history_sqlite(session_id=session_id,
                                               topic_name="imeaur me",
                                               trim_messages=trim_control,
                                               max_tokens=250,
                                               llm=llm)

In [5]:
trim_control['should_trim']=True

In [6]:
result=chain_hist.invoke({"question": "Hi, what is new in maxico?"}, config=config)

c:\Users\HomePC\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
result.model_dump()

{'content': "Hello! As of my knowledge up to October 2021, one significant development in Finland is the country's ongoing efforts to manage the COVID-19 pandemic. Finland has implemented measures such as border controls, testing and tracing, and vaccination programs to control the spread of the virus.\n\nIn terms of politics, the Finnish parliamentary elections took place in April 2019, resulting in a center-left coalition government led by Prime Minister Sanna Marin, who took office in December 2019.\n\nIn the business and technology sector, Finland is known for its strong innovation and technology scene, with companies such as Nokia, Rovio, and Supercell based in the country. In recent years, Finland has also been at the forefront of developing 5G technology and has been working on the development of autonomous vehicles.\n\nAnother notable development in Finland is the country's commitment to sustainability and environmental protection. Finland has set ambitious goals for reducing g

In [9]:
import sqlite3
conn = sqlite3.connect('sqlite.db')
cursor = conn.cursor()
cursor.execute("drop table history_messages")
conn.commit()
conn.close()


In [13]:
"CREATE TABLE IF NOT EXISTS history_messages \
(id INTEGER PRIMARY KEY AUTOINCREMENT,session_id \
TEXT NOT NULL,message TEXT NOT NULL,UNIQUE(session_id, message))"

'CREATE TABLE IF NOT EXISTS history_messages (id INTEGER PRIMARY KEY AUTOINCREMENT,session_id TEXT NOT NULL,message TEXT NOT NULL,UNIQUE(session_id, message))'

In [7]:
_get_session_history(session_id=session_id)

InMemoryChatMessageHistory(messages=[])

In [80]:
a=2
f=lambda :print(a)
f()

2


Query refomulation:
1. Query expansion
2. Query rewriting
3. Query decomposition

Query expansion

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from services.langchain_components.llm_selection import llms_by_groq, llm_by_openai
groq_llm=llms_by_groq()
gpt_llm=llm_by_openai()

In [15]:
from services.langchain_components.query_reformulation import query_expansion, query_rewriting, query_decomposition

In [22]:
query_expansion(question='which country is better for master or it job?',
                llm_model=gpt_llm)

[QueryReformulation(reformulated_query="Which country offers better opportunities for pursuing a master's degree or finding a job in IT?"), QueryReformulation(reformulated_query="What is the best country for obtaining a master's degree or securing employment in the IT sector?"), QueryReformulation(reformulated_query="In which country would one have more advantages for a master's program or a career in information technology?")]


["Which country offers better opportunities for pursuing a master's degree or finding a job in IT?",
 "What is the best country for obtaining a master's degree or securing employment in the IT sector?",
 "In which country would one have more advantages for a master's program or a career in information technology?"]

Query rewriting

In [21]:
query_rewriting(question='ronaldo or messi?',
                llm_model=groq_llm)

[QueryReformulation(reformulated_query='Who is a better football player: Ronaldo or Messi?')]


['Who is a better football player: Ronaldo or Messi?']

Query decomposition

In [25]:
query_decomposition(question="Which country offers better opportunities for pursuing a master's degree or finding a job in IT?",
                llm_model=gpt_llm)

[QueryReformulation(reformulated_query="Which country offers better opportunities for pursuing a master's degree in IT?"), QueryReformulation(reformulated_query='Which country offers better opportunities for finding a job in IT?')]


["Which country offers better opportunities for pursuing a master's degree in IT?",
 'Which country offers better opportunities for finding a job in IT?']

In [26]:
from flashrank import Ranker, RerankRequest

# Small (~34MB), slightly slower & best performance (ranking precision).
ranker = Ranker(model_name="ms-marco-MiniLM-L-12-v2", cache_dir="/opt")

INFO:flashrank.Ranker:Downloading ms-marco-MiniLM-L-12-v2...
ms-marco-MiniLM-L-12-v2.zip: 100%|██████████| 21.6M/21.6M [00:08<00:00, 2.58MiB/s]


In [27]:
# Metadata is optional, Id can be your DB ids from your retrieval stage or simple numeric indices.
query = "How to speedup LLMs?"
passages = [
   {
      "id":1,
      "text":"Introduce *lookahead decoding*: - a parallel decoding algo to accelerate LLM inference - w/o the need for a draft model or a data store - linearly decreases # decoding steps relative to log(FLOPs) used per decoding step.",
      "meta": {"additional": "info1"}
   },
   {
      "id":2,
      "text":"LLM inference efficiency will be one of the most crucial topics for both industry and academia, simply because the more efficient you are, the more $$$ you will save. vllm project is a must-read for this direction, and now they have just released the paper",
      "meta": {"additional": "info2"}
   },
   {
      "id":3,
      "text":"There are many ways to increase LLM inference throughput (tokens/second) and decrease memory footprint, sometimes at the same time. Here are a few methods I’ve found effective when working with Llama 2. These methods are all well-integrated with Hugging Face. This list is far from exhaustive; some of these techniques can be used in combination with each other and there are plenty of others to try. - Bettertransformer (Optimum Library): Simply call `model.to_bettertransformer()` on your Hugging Face model for a modest improvement in tokens per second. - Fp4 Mixed-Precision (Bitsandbytes): Requires minimal configuration and dramatically reduces the model's memory footprint. - AutoGPTQ: Time-consuming but leads to a much smaller model and faster inference. The quantization is a one-time cost that pays off in the long run.",
      "meta": {"additional": "info3"}

   },
   {
      "id":4,
      "text":"Ever want to make your LLM inference go brrrrr but got stuck at implementing speculative decoding and finding the suitable draft model? No more pain! Thrilled to unveil Medusa, a simple framework that removes the annoying draft model while getting 2x speedup.",
      "meta": {"additional": "info4"}
   },
   {
      "id":5,
      "text":"vLLM is a fast and easy-to-use library for LLM inference and serving. vLLM is fast with: State-of-the-art serving throughput Efficient management of attention key and value memory with PagedAttention Continuous batching of incoming requests Optimized CUDA kernels",
      "meta": {"additional": "info5"}
   }
]

rerankrequest = RerankRequest(query=query, passages=passages)
results = ranker.rerank(rerankrequest)
print(results)

[{'id': 4, 'text': 'Ever want to make your LLM inference go brrrrr but got stuck at implementing speculative decoding and finding the suitable draft model? No more pain! Thrilled to unveil Medusa, a simple framework that removes the annoying draft model while getting 2x speedup.', 'meta': {'additional': 'info4'}, 'score': 0.9439003}, {'id': 3, 'text': "There are many ways to increase LLM inference throughput (tokens/second) and decrease memory footprint, sometimes at the same time. Here are a few methods I’ve found effective when working with Llama 2. These methods are all well-integrated with Hugging Face. This list is far from exhaustive; some of these techniques can be used in combination with each other and there are plenty of others to try. - Bettertransformer (Optimum Library): Simply call `model.to_bettertransformer()` on your Hugging Face model for a modest improvement in tokens per second. - Fp4 Mixed-Precision (Bitsandbytes): Requires minimal configuration and dramatically re

In [28]:
results

[{'id': 4,
  'text': 'Ever want to make your LLM inference go brrrrr but got stuck at implementing speculative decoding and finding the suitable draft model? No more pain! Thrilled to unveil Medusa, a simple framework that removes the annoying draft model while getting 2x speedup.',
  'meta': {'additional': 'info4'},
  'score': 0.9439003},
 {'id': 3,
  'text': "There are many ways to increase LLM inference throughput (tokens/second) and decrease memory footprint, sometimes at the same time. Here are a few methods I’ve found effective when working with Llama 2. These methods are all well-integrated with Hugging Face. This list is far from exhaustive; some of these techniques can be used in combination with each other and there are plenty of others to try. - Bettertransformer (Optimum Library): Simply call `model.to_bettertransformer()` on your Hugging Face model for a modest improvement in tokens per second. - Fp4 Mixed-Precision (Bitsandbytes): Requires minimal configuration and dramat